In [1]:
from bs4 import BeautifulSoup
import urllib.request
import requests
from pprint import pprint
import pandas as pd
import numpy as np
import json
import copy
from datetime import datetime, timezone
from google.cloud import storage
import gcsfs
import re
from sqlalchemy import create_engine, text
from typing import List

pd.set_option('display.max_columns', None)

In [2]:
postgres_config = {
    "host": "findy-medium-stage.czmgcqkw4ett.ap-southeast-1.rds.amazonaws.com",
    "database": "findy_medium_stage",
    "user": "postgres",
    "password": "F!nDy!Med!umStage2o24",
    "port": "5432"
}

In [3]:
def build_upsert_query(cols: List[str],
                       table_name: str,
                       unique_key: List[str]=[],
                       cols_not_for_update: List[str] = None) -> str:
    """
    Builds postgres upsert query using input arguments.
    Note: In the absence of unique_key, this will be just an insert query.
    Example : build_upsert_query(
        ['col1', 'col2', 'col3', 'col4'],
        "my_table",
        ['col1'],
        ['col2']
    ) ->
    INSERT INTO my_table (col1, col2, col3, col4) VALUES %s
    ON CONFLICT (col1) DO UPDATE SET (col3, col4) = (EXCLUDED.col3, EXCLUDED.col4) ;
    :param cols: the postgres table columns required in the
        insert part of the query.
    :param table_name: the postgres table name.
    :param unique_key: unique_key of the postgres table for checking
        unique constraint violations.
    :param cols_not_for_update: columns in cols which are not required in
        the update part of upsert query.
    :return: Upsert query as per input arguments.
    """
    cols = [f'"{col}"' for col in cols]
    cols_str = ', '.join(cols)
    insert_query = """ INSERT INTO %s (%s) VALUES %%s """ % (
        table_name, cols_str
    )
    if cols_not_for_update is not None:
        cols_not_for_update.extend(unique_key)
    else:
        cols_not_for_update = [col for col in unique_key]
    cols_not_for_update = [f'"{col}"' for col in cols_not_for_update]
    unique_key = [f'"{col}"' for col in unique_key]
    unique_key_str = ', '.join(unique_key)

    update_cols = [f"{col}" for col in cols if col not in cols_not_for_update]
    update_cols_str = ', '.join(update_cols)
    update_cols_with_excluded_markers = [f'EXCLUDED.{col}' for col in update_cols]
    update_cols_with_excluded_markers_str = ', '.join(
        update_cols_with_excluded_markers
    )
    if len(update_cols) > 1:
        equality_clause = "(%s) = (%s)"
    else:
        equality_clause = "%s = %s"

    on_conflict_clause = f""" ON CONFLICT (%s) DO UPDATE SET {equality_clause} ;"""
    on_conflict_clause = on_conflict_clause % (
        unique_key_str,
        update_cols_str,
        update_cols_with_excluded_markers_str
    )
    if len(unique_key) == 0:
        return insert_query
    return insert_query + on_conflict_clause

In [4]:
storage_client = storage.Client()
today = datetime.now()
table_name = "douyin_video"
api_name = "search_for_star_hot_item"
bucket_name = "3_staging_area"

In [5]:
processing_blobs = [
{
    "blob": blob,
    "date": blob.name.split('/')[2],
    "batch": int(blob.name.split('/')[-1].replace(".parquet", "").split("_")[-1]),
} for blob in storage_client.list_blobs("3_staging_area",prefix="1_xingtu/douyin_video/") if api_name in blob.name]
processing_blobs

[{'blob': <Blob: 3_staging_area, 1_xingtu/douyin_video/2024-09-30/search_for_star_hot_item_241001_0.parquet, 1727772493257816>,
  'date': '2024-09-30',
  'batch': 0},
 {'blob': <Blob: 3_staging_area, 1_xingtu/douyin_video/2024-10-02/search_for_star_hot_item_241002_0.parquet, 1727845663822959>,
  'date': '2024-10-02',
  'batch': 0},
 {'blob': <Blob: 3_staging_area, 1_xingtu/douyin_video/2024-10-06/search_for_star_hot_item_241007_0.parquet, 1728275199308829>,
  'date': '2024-10-06',
  'batch': 0}]

In [6]:
bucket = storage_client.get_bucket(bucket_name)
meta_blob = bucket.blob("1_xingtu/douyin_video/meta.json")
if meta_blob.exists():
    processed_blobs = json.loads(meta_blob.download_as_string())
else:
    processed_blobs = []
processed_blobs


[{'file_path': 'gs://3_staging_area/1_xingtu/douyin_video/2024-09-30/search_for_star_hot_item_241001_0.parquet',
  'date': '2024-09-30',
  'batch': 0},
 {'file_path': 'gs://3_staging_area/1_xingtu/douyin_video/2024-10-02/search_for_star_hot_item_241002_0.parquet',
  'date': '2024-10-02',
  'batch': 0}]

In [7]:
to_process = []
for processing_blob in processing_blobs:
    if processing_blob["batch"] not in [processed_blob["batch"] for processed_blob in processed_blobs if api_name in processed_blob["file_path"] and processing_blob["date"] == processed_blob["date"]]:
        to_process.append(processing_blob)
pprint(to_process)
print(len(to_process))

[{'batch': 0,
  'blob': <Blob: 3_staging_area, 1_xingtu/douyin_video/2024-10-06/search_for_star_hot_item_241007_0.parquet, 1728275199308829>,
  'date': '2024-10-06'}]
1


In [8]:
def remove_bracket_duplicated(value):
    if len(value) > 6:
        return str(value).replace("{'{", "{").replace("}'}", "}")
    else:
        return value

In [12]:
for item in to_process:
    engine = create_engine("postgresql+psycopg2://{user}:{password}@{host}:{port}/{database}".format(**postgres_config))
    l_df = pd.read_sql_table("douyin_video",con=engine)
    m_df = pd.read_sql_table("douyin_influencer",con=engine)
    r_df = pd.read_parquet("gs://" + bucket_name + "/" + item["blob"].name)
    r_df["influencer_id"] = r_df["influencer_id"].astype(str)
    df = pd.merge(m_df, r_df, left_on="user_id", right_on="influencer_id", how="inner")
    df.columns = [column.replace("_y", "") for column in df.columns]
    df = df[["id"] + list(r_df.columns)]
    upsert_df = df.drop("influencer_id", axis=1)
    upsert_df = upsert_df.rename(columns={"id": "influencer_id"})
    upsert_df["sale_amount"] = upsert_df["sale_amount"].astype(float).astype(int)
    upsert_df["comment"] = 0
    upsert_df["share"] = 0
    upsert_df["create_time"] = str(today)
    upsert_df["is_brand"] = False
    upsert_df["duration"] = 0
    upsert_df["updated"] = str(today)
    upsert_df["title"] = upsert_df["title"].str.replace("'", "''")
    for item_id in upsert_df[upsert_df["item_id"].duplicated()]["item_id"]:
        xt_ranking_type = str({value for value in upsert_df[upsert_df["item_id"] == item_id]["xt_ranking_type"]})
        upsert_df.loc[upsert_df["item_id"] == item_id, "xt_ranking_type"] = xt_ranking_type
    upsert_df.loc[~upsert_df["item_id"].duplicated(keep=False), ["xt_ranking_type"]] = "{" + upsert_df[~upsert_df["item_id"].duplicated(keep=False)]["xt_ranking_type"].astype(str) + "}"
    upsert_df[~upsert_df["item_id"].duplicated(keep=False)]
    upsert_df["xt_ranking_type"] = upsert_df["xt_ranking_type"].apply(remove_bracket_duplicated)
    upsert_df = upsert_df.drop_duplicates()
    upsert_df = upsert_df.drop_duplicates(subset="item_id")
    query = build_upsert_query(upsert_df.columns, "douyin_video", ["item_id"])
    value = ", ".join([str(record).replace('"', "'") for record in upsert_df.to_records(index=False)])
    query = query % value
    with engine.connect() as conn:
        result = conn.execute(text(query))
        conn.commit()
    processed_blobs.append({
        'file_path': "gs://" + bucket_name + "/" + item["blob"].name,
        'date': item["date"],
        'batch': item["batch"]
    })
    meta_blob.upload_from_string(json.dumps(processed_blobs))

/var/folders/b0/35cpxwfx6134b2y90f_1cr0m0000gn/T/ipykernel_35027/3089951594.py:22: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '{1, 3}' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  upsert_df.loc[upsert_df["item_id"] == item_id, "xt_ranking_type"] = xt_ranking_type
